In [0]:
%sh

echo "host = <your host>" >> ~/.databrickscfg




Error: InvalidConfigurationError: You haven't configured the CLI yet! Please configure by entering `/databricks/python3/bin/databricks configure`


In [0]:
clientId = dbutils.secrets.get(scope='all', key='clientId')
clientSecret = dbutils.secrets.get(scope='all', key='clientSecret')
storageKey = dbutils.secrets.get(scope='all', key='strgAIMLKey')

tenantId = '16b3c013-d300-468d-ac64-7eda0820b6d3'

entraOAuthTokenUrl = f'https://login.microsoftonline.com/{tenantId}/oauth2/token"'
mountPoint = "/mnt/data/adls"
containerName = 'databricks'
storageName = 'strgazdl'
abfss = f"abfss://{containerName}@{storageName}.dfs.core.windows.net/"

adlsMountedDataPath = 'dbfs:/mnt/data/adls/data'

print(storageKey)
print(clientSecret)

[REDACTED]
[REDACTED]


In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": clientId,
          "fs.azure.account.oauth2.client.secret": clientSecret,
          "fs.azure.account.oauth2.client.endpoint": entraOAuthTokenUrl}

# mount using ABFS using service principle, not successful, 401 error
# if not any(mount == mountPoint for mount in dbutils.fs.mounts()):
#   dbutils.fs.mount(
#     source = abfss,
#     mount_point = mountPoint,
#     extra_configs = configs)


# mount using storage using storage key, successful
# https://learn.microsoft.com/en-us/azure/databricks/dbfs/mounts#how-does-azure-databricks-mount-cloud-object-storage
dbutils.fs.mount(
  source = f"wasbs://{containerName}@{storageName}.blob.core.windows.net",
  mount_point = mountPoint,
  extra_configs = {f"fs.azure.account.key.{storageName}.blob.core.windows.net": storageKey})




True

In [0]:
%fs ls dbfs:/mnt/data/adls/data

path,name,size,modificationTime
dbfs:/mnt/data/adls/data/address.parquet,address.parquet,622007,1711693940000
dbfs:/mnt/data/adls/data/customer.json,customer.json,40315809,1711693942000
dbfs:/mnt/data/adls/data/web_sales.csv,web_sales.csv,146169257,1711693945000


In [0]:
# Generate SQL query
sql_query = """
SELECT country, state, city from address  Limit 10
"""

# Execute SQL query
result_df = spark.sql(sql_query)

# Display result dataframe
display(result_df)